# Modules

In [1]:

import pandas as pd
import numpy as np
import numpy.random as npr 
import matplotlib.pyplot as plt
from tqdm import tqdm


from sklearn.metrics import accuracy_score

from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor


# Création d'un dataset et fonction de coût

In [2]:
K = 2
# Créez un ensemble de données avec 1000 échantillons, 20 caractéristiques et 2 classes pour la variable cible
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, n_classes=K, random_state=0)
# n_informative : nombre de caractéristiques informatives, celles qui sont réellement utiles pour prédire la classe cible.
# n_redundant : nombre de caractéristiques redondantes générées comme des combinaisons linéaires des caractéristiques informatives.

y[y==0] = -1 #permet de mettre -1 à la place de 0

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2)

$$loss\_reg\_log(z, y)= \dfrac{1}{n}\sum_{i=1}^n \log(1 + \exp(-y_i \times z_i))$$

$$\nabla loss\_reg\_log(z, y)_j= \dfrac{1}{n}\times  \dfrac{-y_j \times \exp(-y_j \times z_j)}{1 + \exp(-y_j \times z_j)}$$

In [3]:
## Ici z correspond à F(X) un vecteur de \R^n
def loss_reg_log(z, y):
    return np.mean(np.log(1 + np.exp(-y * z)))

def grad_loss_reg_log(z, y):
    return (-y * np.exp(-y * z) / (1 + np.exp(-y * z))) * 1/len(y)
    

# Initialisation du modèle. 

On commence par un modèle simple comme un arbre de décision peu profond. Ici un DecisionTreeRegressor est utilisé car dans le Gradient Boosting, même pour la classification, les modèles sont souvent entraînés pour prédire des modifications de probabilité plutôt que des classes directes.

In [4]:
# Initialisation du modèle de base - un arbre de décision simple - 
#a stump : Une souche de décision est un modèle d'apprentissage automatique composé d'un arbre de décision à un niveau.
base_model = DecisionTreeRegressor(max_depth=4)

# Entraînement du modèle de base sur l'ensemble d'entraînement
base_model.fit(X_train, Y_train)

# Utilisation du modèle de base pour faire des prédictions initiales
predictions_init_train = base_model.predict(X_train)
predictions_init_test = base_model.predict(X_test)

In [5]:
print('accuracy train = ' , accuracy_score(Y_train, np.sign(predictions_init_train)))
print('accuracy test = ' , accuracy_score(Y_test, np.sign(predictions_init_test)))

accuracy train =  0.79125
accuracy test =  0.66


# Boucle d'apprentissage :

* Définir le Nombre d'Itérations :

Commencer par définir le nombre d'itérations ou arbres dans le modèle de boosting $n_estimators$. Utiliser les prédictions du modèle de base comme point de départ.

* Boucle d'Apprentissage :

- Prédiction : Utiliser le modèle actuel pour prédire les classes sur l'ensemble d'entraînement.
- Calcul du Gradient : calculer le gradient de la fonction de perte par rapport aux prédictions, cela donne l'erreur résiduelle que le prochain modèle essaiera de prédire.
- Ajustement du Nouveau Modèle : entraîner un nouveau modèle faible pour prédire ces gradients.
- Mise à Jour des Prédictions : mettre à jour les prédictions globales en ajoutant les prédictions du nouveau modèle, pondérées par un taux d'apprentissage.

In [6]:
n_estimators = 100 # Nombre d'itérations
learning_rate = 2  # Taux d'apprentissage
predictions_train = predictions_init_train
predictions_test = predictions_init_test

for i in range(n_estimators):
    
    # a. Prédiction avec le modèle actuel
    current_prediction_train = predictions_train

    # b. Calcul du gradient de la fonction de perte
    residuals = - grad_loss_reg_log(Y_train, current_prediction_train)  # gradient_of_loss_function doit être défini

    # c. Entraînement d'un nouveau modèle sur les résidus
    new_model = DecisionTreeRegressor(max_depth=4)
    new_model.fit(X_train, residuals)

    # d. Mise à jour des prédictions
    predictions_train += learning_rate * new_model.predict(X_train)
    print(predictions_train[0])
    predictions_test += learning_rate * new_model.predict(X_test)
    
    print(f'accuracy train step {i+1} = ' , accuracy_score(Y_train, np.sign(predictions_train)))
    print(f'accuracy test step {i+1} = ' , accuracy_score(Y_test, np.sign(predictions_test)))

-0.8468426293658605
accuracy train step 1 =  0.79125
accuracy test step 1 =  0.66
-0.847531705831603
accuracy train step 2 =  0.79125
accuracy test step 2 =  0.66
-0.8482210753143655
accuracy train step 3 =  0.79125
accuracy test step 3 =  0.66
-0.8489107375770909
accuracy train step 4 =  0.79125
accuracy test step 4 =  0.66
-0.8496006923823737
accuracy train step 5 =  0.79125
accuracy test step 5 =  0.66
-0.8502909394924609
accuracy train step 6 =  0.79125
accuracy test step 6 =  0.66
-0.8509814786692528
accuracy train step 7 =  0.79125
accuracy test step 7 =  0.66
-0.8516723096743033
accuracy train step 8 =  0.79125
accuracy test step 8 =  0.66
-0.8523634322688214
accuracy train step 9 =  0.79125
accuracy test step 9 =  0.66
-0.8530548462136712
accuracy train step 10 =  0.79125
accuracy test step 10 =  0.66
-0.8537465512693735
accuracy train step 11 =  0.79125
accuracy test step 11 =  0.66
-0.8544385471961056
accuracy train step 12 =  0.79125
accuracy test step 12 =  0.66
-0.85513083

# Comparaison avec une implémentation du Gradient Boosting du type AdaBoost

In [158]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Création du modèle AdaBoost avec un arbre de décision comme modèle de base
ada_model = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=4),
    n_estimators=100,
    learning_rate=0.1,
    random_state=0
)

# Entraînement du modèle AdaBoost
ada_model.fit(X_train, Y_train)

# Prédiction sur l'ensemble de train
ada_predictions_train = ada_model.predict(X_train)
# Prédiction sur l'ensemble de test
ada_predictions_test = ada_model.predict(X_test)


# Évaluation de la précision
ada_accuracy_train = accuracy_score(Y_train, ada_predictions_train)
print(f'AdaBoost Accuracy train: {ada_accuracy_train}')
# Évaluation de la précision
ada_accuracy_test = accuracy_score(Y_test, ada_predictions_test)
print(f'AdaBoost Accuracy test: {ada_accuracy_test}')

/home/ibotcazou/.local/lib/python3.10/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoost Accuracy train: 1.0
AdaBoost Accuracy test: 0.845
